In [0]:
CREATE OR REFRESH STREAMING TABLE bronzee_drivers
COMMENT 'Raw customers data ingested from the source system operational data'
TBLPROPERTIES ('quality' = 'bronze')
AS 
SELECT *,
       _metadata.file_path AS input_file_path,
       CURRENT_TIMESTAMP AS ingestion_timestamp
FROM cloud_files(
  '/Volumes/ishwariformulabox/schemalanding/volume2/drivers.csv',
  'csv',
  map('cloudFiles.inferColumnTypes', 'true')
);

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_drivers (
    CONSTRAINT valid_driver_id EXPECT (driverId IS NOT NULL AND driverId > 0),
    CONSTRAINT valid_driver_name EXPECT (forename IS NOT NULL AND surname IS NOT NULL),
    CONSTRAINT valid_code EXPECT (code IS NOT NULL)
)
COMMENT "Cleaned drivers data with easy transformations"
TBLPROPERTIES ("quality" = "silver")
AS
SELECT
    driverId,
    driverRef,
    number,
    code,
    forename,
    surname,
    dob,
    nationality,
    url,
 
    CONCAT(forename, ' ', surname) AS full_name,
    YEAR(CAST(dob AS DATE)) AS birth_year,
    CASE
        WHEN nationality IN ('British', 'American', 'Canadian') THEN 'North America/UK'
        WHEN nationality IN ('German', 'French', 'Italian', 'Spanish') THEN 'Europe'
        ELSE 'Other'
    END AS region_category,
    CURRENT_TIMESTAMP() AS processed_at
FROM STREAM(LIVE.bronzee_drivers);